# Kepler's equation

## Ellipse Parameterizations

![elip](https://robot.inf.um.es/material/demos/elipkepler.svg)

Excentricidad:

$$ e = \sqrt{1 - \frac{b^2}{a^2}} $$

**True anomaly**: cordenadas polares de una elipse desde un foco:

$$ r = \frac{a(1-e^2)}{1+e\cos(\nu)} $$

**Eccentric anomaly**: coordenadas polares desde el centro:

$$r = \sqrt{a^2 + (b^2-a^2)\sin^2(E)} $$

que se simplifica a algo tan simple como:

$$ x = a \cos(E) \hspace{1cm} y=b \sin(E) $$

Pero ojo, el parámetro $E$ no es el ángulo geométrico al punto.

Finalmente, está la **mean anomaly** $M$, una parametrización del tiempo, relacionada con la posición real $E$ mediante la ecuación de Kepler.

In [ ]:
import numpy as np, matplotlib.pyplot as plt

nu = np.linspace(0,2*np.pi, 13)
a = 1
e = 0.5
r = a*(1-e**2)/(1+e*np.cos(nu))
x = r*np.cos(nu)
y = r*np.sin(nu)
plt.plot(x,y,'.-',markersize=10);
for x,y in zip(x,y):
    plt.plot([0,x],[0,y],color='gray')

E = np.linspace(0,2*np.pi, 25)
b = a * np.sqrt(1-e**2)
x = a*np.cos(E) - a*e
y = b*np.sin(E)
plt.plot(x,y,'.',markersize=3,color='black');
plt.plot(x,y,markersize=3,color='gray',lw=0.5);
for x,y in zip(x,y):
    plt.plot([-a*e,x],[0,y],color='gray',lw=0.5)    
    
plt.plot([0][0],'.',markersize=20);
plt.plot([-a*e],[0],'.',markersize=15,color='black');
plt.axis('equal');

## Kepler's equation

$$M = E - e \sin{E}$$

Un problema fundamental es la solución analítica de una órbita kepleriana.

Obviamente, mediante integración numérica podemos calcular la posición y velocidad siguiente de los cuerpos en un sistema gravitatorio, y por supuesto también en el caso más simple de 2 cuerpos, uno de ellos de masa despreciable.

Las leyes de Kepler, incialmente obtenidas de forma empírica y posteriormente deducidas a partir de las leyes de gravitación de Newton, nos dan la forma de la órbita (elipse) y la variación de velocidad en ella (momento angular constante), pero no tenemos una dependencia explícita de la posición respecto al tiempo.

Cuando se lleva el problema a la configuración geométrica más simple, a lo más que podemos llegar es una relación transcendental entre (parametrizaciones adecuadas) del tiempo y la posición. Esta dependencia puede resolverse fácilmente mediante métodos numéricos, basados en refinar progresivamente un resultado tentativo.

## Graphic solution

In [ ]:
plt.figure(figsize=(5,7))
E = np.linspace(0, 2*np.pi, 100)
plt.plot(E,E,label='$x$')
plt.plot(E,np.sin(E),label='$\sin x$')
plt.plot(E,E-0.3*np.sin(E),label="$x-0.3\sin x$")
plt.grid()
plt.legend()
plt.axis('equal');

In [ ]:
import scipy, kepler

In [ ]:
%psource kepler.anomM2E

In [ ]:
x = kepler.anomM2E(0.4, 0.3)
x

In [ ]:
x - 0.3*np.sin(x)

## Numerical integration

In [ ]:
import numpy as np
import numpy.linalg as la 
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
def accel(x,m):
    n = len(m)
    a = np.zeros([n,3])
    for k in range(n):
        for j in range(n):
            if k != j:
                r = x[j]-x[k]
                r3 = la.norm(r)**3
                a[k] += m[j] / r3 * r
    return a

In [ ]:
def nbody(r0,v0,m,t):
    n = len(m)
    
    def xdot(z,t):
        #print(len(z))
        global count
        count += 1
        r = z[:3*n].reshape(-1,3)
        v = z[3*n:]
        a = accel(r,m).flatten()
        return np.concatenate([v,a])
    
    s0 = np.concatenate([r0.flatten(),v0.flatten()])
    #print(s0)
    s = odeint(xdot,s0,t)
    
    return [(s[:3*n].reshape(-1,3), s[3*n:].reshape(-1,3)) for s in s]

https://en.wikipedia.org/wiki/Standard_gravitational_parameter

In [ ]:
ua = 150e9
yr = 365*24*60*60

In [ ]:
datar = np.array(np.matrix("""
0      0   0   0   0        0    1.327E20    ;
0.466  0   0   0   47E3     0    2.203E13    ;
0.723  0   0   0   35E3     0    3.249E14    ;
1      0   0   0   30.0E3   0    3.986E14    ;
1.0026 0   0   0   31.0E3   0    4.905E12    ;
1.52   0   0   0   24E3     0    4.283E13    ;
5.2    0   0   0   13E3     0    1.267E17    ;
9.5    0   0   0   9.7E3    0    3.793E16    ;
19     0   0   0   6.8E3    0    5.794E15    ;
30     0   0   0   5.43E3   0    6.837E15    ;
1      0   0   0   15.0E3   0    1E10        """))

In [ ]:
rotation = kepler.myrotation

In [ ]:
data = datar[[0,-1]]

r0 = data[:,:3]
v0 = data[:,3:6] * yr    / ua
mu = data[:,6]   * yr**2 / ua**3

r0[1] = rotation((0,0,1),np.radians(-45)) @ r0[1]
v0[1] = rotation((0,0,1),np.radians(-45)) @ rotation((1,0,0),np.radians(30)) @ v0[1]

In [ ]:
dt = 0.435/20
N = 15

count = 0
simul = nbody(r0,v0,mu,np.arange(N+1)*dt)
print(count)

sun,earth = np.array([x[0] for x in simul]).transpose(1,2,0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

ax.plot(*sun,'.',color='orange',markersize=15)
ax.plot(*earth,'.-',color='red')

ax.set_xlim(-0.5,0.5); ax.set_xlabel('x')
ax.set_ylim(-0.5,0.5); ax.set_ylabel('y')
ax.set_zlim(-0.5,0.5); ax.set_zlabel('z');

## Closed form

In [ ]:
a,e,om,OM,i,M0 = orbit = kepler.keplerElements(r0[1],v0[1],mu[0])
print(orbit)
T = kepler.period(a,mu[0])
print(M0)
print(T)

In [ ]:
def p_v_kepler(t):
    M = M0  + t/T*2*np.pi
    return kepler.kepler2stv(a,e,om,OM,i,M,mu[0])

In [ ]:
p2 = np.array([p_v_kepler(t)[0] for t in np.arange(N+1)*dt])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

plt.plot(*p2.T,lw=5,alpha=0.5,color='green')

ax.plot(*sun,'.',color='orange',markersize=15)
ax.plot(*earth,'.-',color='red')

ax.set_xlim(-0.5,0.5); ax.set_xlabel('x')
ax.set_ylim(-0.5,0.5); ax.set_ylabel('y')
ax.set_zlim(-0.5,0.5); ax.set_zlabel('z');

## Derivation

En wikipedia hay una [deducción geométrica](https://en.wikipedia.org/wiki/Kepler%27s_laws_of_planetary_motion#Mean_anomaly,_M) sencilla.

(Lo que sigue puede no tener mucho sentido.)

Otra posibilidad es atacar por [separación de variables](https://en.wikipedia.org/wiki/Separation_of_variables):

$$\frac{d\theta}{dt} = k (1+e\cos\theta)^2$$

$$\frac{1}{(1+e\cos\theta)^2} d\theta = k dt$$

Esa integral se encuentra en tablas o wolfram alpha, aunque es bastante compleja. Es lo que hace Curtis.

Recordemos que, por geometría, tenemos tanto $r$ como $\theta$ en función de $E$.

$$\tan \frac{\theta}{2} = \sqrt{\frac{1+e}{1-e}}\tan\frac{E}{2}$$

$$r = a (1 - e \cos E)$$


Por tanto en la condición de velocidad areolar constante se puede aplicar un cambio de variable para tener como incógnita $E(t)$. A ver si hay más suerte...

$$r^2 \frac{d\theta}{dt} = r^2 \frac{d\theta}{dE} \frac{dE}{dt} = n a b$$

$$\theta = 2\arctan\left(\rho \tan\frac{E}{2}\right)$$

$$\frac{d\theta}{dE} = \frac{\rho}{\rho^{2} \sin^{2}{\left (\frac{E}{2} \right )} + \cos^{2}{\left (\frac{E}{2} \right )}}$$

La integral que sale también es muy complicada y poco práctica, ya que aparentemente no se simplifica el $r^2$ que multiplicará después.

Lo que hace Orús et al. es relacionar los diferenciales:

$$\frac{1}{2}\frac{1}{\cos^2\frac{\theta}{2}}d\theta = \rho \frac{1}{2}\frac{1}{\cos^2\frac{E}{2}}dE$$

$$d\theta = \rho \frac{\cos^2\frac{\theta}{2}}{\cos^2\frac{E}{2}}dE$$

Con la ventaja de que no hay que sustituir la transformación complicada con tangentes y arcotangentes de $\theta(E)$ ya que tenemos una expresión compacta para el

$$\cos^2\frac{\theta}{2} = \frac{a(1-e)}{r}\cos^2\frac{E}{2}$$

que simplifica muchísimo la derivada buscada:

$$d\theta = \frac{\rho a (1-e)}{r} dE = \sqrt\frac{1+e}{1-e}\frac{ a (1-e)}{r} dE = \sqrt{(1+e)(1-e)} \;\frac{a}{r} dE = \frac{b}{r}dE$$

La velocidad areolar expresada con $E$ queda muy simple:

$$nabdt = r^2 d\theta = r b dE = b  a (1 - e \cos E) dE$$

Integrando en ambos lados:

$$nt \equiv M = E - e \sin E$$

(Se introduce una medida del tiempo normalizada respecto al período, la *anomalía media*, expresada como el ángulo recorrido.)

## Ecuación diferencial para $\theta(t)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.integrate import odeint

In [ ]:
mu = 1
h = 2
e = 0.5
p = h**2/mu
eps = 0.5*mu**2/h**2*(e**2-1)
a = -mu/2/eps
b = a*np.sqrt(1-e**2)
T = np.sqrt(4*np.pi**2*a**3/mu)
n = 2*np.pi/T

In [ ]:
def sdot(v,t):
    return mu**2/h**3*(1+e*np.cos(v))**2

V = np.linspace(0,2*np.pi,100)


plt.plot(V,sdot(V,0));

In [ ]:
t = np.linspace(0,2*T,100)

v = odeint(sdot,0,t)

plt.plot(t,v);

In [ ]:
import scipy.optimize as opt

def eccenAnom(M,e):
    return opt.fsolve(lambda E: E - e*np.sin(E) - M, M)

def trueAnom(E,e):
    # vale para un periodo
    k = np.sqrt((1+e)/(1-e))
    return 2*np.arctan2( k*np.sin(E/2), np.cos(E/2))

In [ ]:
M = np.linspace(0,1*2*np.pi,50)
E = np.array([eccenAnom(m,e) for m in M])
u = [trueAnom(x,e) for x in E]

In [ ]:
plt.plot(M,E,label='eccentric');
plt.plot(M,u,label='true');
plt.xlabel('M (rad)')
plt.ylabel('anomaly (rad)')
plt.legend();

In [ ]:
tm = M/n

In [ ]:
plt.plot(tm,u,lw=8,label='eq. diff.');
plt.plot(t[t<T],v[t<T],color='red',label='kepler');
plt.xlabel('t (s)')
plt.ylabel('true anomaly (rad)')
plt.legend();

In [ ]:
r = a*(1-e*np.cos(E))

plt.plot(E, b/r,lw=8);

rho = np.sqrt((1+e)/(1-e))

cosa = rho/(rho**2 * np.sin(E/2)**2  +  np.cos(E/2)**2)

plt.plot(E,cosa);
plt.xlabel('E'); plt.title('dv/dE');

## References

https://en.wikipedia.org/wiki/Orbital_elements

https://downloads.rene-schwarz.com/download/M002-Cartesian_State_Vectors_to_Keplerian_Orbit_Elements.pdf

https://downloads.rene-schwarz.com/download/M001-Keplerian_Orbit_Elements_to_Cartesian_State_Vectors.pdf

https://en.wikipedia.org/wiki/Eccentricity_vector que puede expresarse con el semieje mayor.

http://www.stardust2013.eu/Portals/63/Images/Training/OTS%20Repository/gronchi_OTS2013.pdf